### Import Libraries

In [126]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
%matplotlib inline


import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px

import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

### Read Data File : Data_PV_SP_IMP.xlsx

In [127]:
df = pd.read_excel('/home/shiva/Documents/UTS/02_Courses/DVN/DVN/AT2 Part1/Data/Data PV SP IMP.xlsx')

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1176 non-null   datetime64[ns]
 1   Hour        1176 non-null   int64         
 2   Page Views  1176 non-null   int64         
 3   Weekday     1176 non-null   object        
 4   Month       1176 non-null   object        
 5   NetA_imp    1176 non-null   int64         
 6   NetA_spd    1176 non-null   int64         
 7   NetB_imp    1176 non-null   int64         
 8   NetB_spd    1176 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 82.8+ KB


### Parse Date

In [129]:
df['Date']=df.Date.apply(lambda x: x.strftime(format="%Y-%m-%d"))

In [130]:
df['Date'].unique()

array(['2019-07-07', '2019-07-08', '2019-07-09', '2019-07-10',
       '2019-07-11', '2019-07-12', '2019-07-13', '2019-07-14',
       '2019-07-15', '2019-07-16', '2019-07-17', '2019-07-18',
       '2019-07-19', '2019-07-20', '2019-07-21', '2019-07-22',
       '2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26',
       '2019-07-27', '2019-07-28', '2019-07-29', '2019-07-30',
       '2019-07-31', '2019-08-01', '2019-08-02', '2019-08-03',
       '2019-08-04', '2019-08-05', '2019-08-06', '2019-08-07',
       '2019-08-08', '2019-08-09', '2019-08-10', '2019-08-11',
       '2019-08-12', '2019-08-13', '2019-08-14', '2019-08-15',
       '2019-08-16', '2019-08-17', '2019-08-18', '2019-08-19',
       '2019-08-20', '2019-08-21', '2019-08-22', '2019-08-23',
       '2019-08-24'], dtype=object)

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        1176 non-null   object
 1   Hour        1176 non-null   int64 
 2   Page Views  1176 non-null   int64 
 3   Weekday     1176 non-null   object
 4   Month       1176 non-null   object
 5   NetA_imp    1176 non-null   int64 
 6   NetA_spd    1176 non-null   int64 
 7   NetB_imp    1176 non-null   int64 
 8   NetB_spd    1176 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 82.8+ KB


### Read Daypart file into dataframe

In [132]:
df_daypart = pd.read_excel('/home/shiva/Documents/UTS/02_Courses/DVN/DVN/AT2 Part1/Data/Daypart.xlsx')
df_daypart

,Hour,Day Part
0,0,Late night
1,1,Late night
2,2,Late night
3,3,Late night
4,4,Early Morning
5,5,Early Morning
6,6,Early Morning
7,7,Morning
8,8,Morning
9,9,Morning


In [133]:
cats = [ 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

cat_type = CategoricalDtype(categories=cats, ordered=True)
df['Weekday'] = df['Weekday'].astype(cat_type)


#df['Weekday'] = df['Weekday'].astype('category', categories=cats, ordered=True)

### Prepare data for plotting - Summarise Weekly Avg's

In [134]:
df_plot=df.groupby(['Weekday'])[['Page Views', 'NetA_imp','NetA_spd'\
                                         , 'NetB_imp', 'NetB_spd']].mean().round(2).reset_index()

In [135]:
df_plot.head()

,Weekday,Page Views,NetA_imp,NetA_spd,NetB_imp,NetB_spd
0,Mon,159.83,7.29,171.46,5.98,149.20
1,Tue,141.18,6.74,56.57,7.85,174.62
2,Wed,141.37,3.76,88.20,6.22,117.10
3,Thu,134.17,5.23,162.33,4.33,87.55
4,Fri,128.02,2.69,85.91,4.82,104.48


### Prepare data for plotting - Summarise by Date(Sum)

In [136]:
df_daily_agg=df.groupby(['Date'])[['Page Views', 'NetA_imp','NetA_spd'\
                                         , 'NetB_imp', 'NetB_spd']].sum().reset_index()

df_daily_agg_aug = df_daily_agg[df_daily_agg['Date'] >= '2019-08-01']

df_daily_agg_aug.head()

,Date,Page Views,NetA_imp,NetA_spd,NetB_imp,NetB_spd
25,2019-08-01,2783,2,137,33,913
26,2019-08-02,2658,40,823,0,0
27,2019-08-03,2748,21,405,42,1170
28,2019-08-04,3000,83,2134,56,2018
29,2019-08-05,3256,146,4025,218,7783


### Q1 - Total Impressions For Aug 2019

In [137]:
fig = go.Figure(data=[go.Bar(name="NetA_imp",
            x=df_daily_agg_aug['Date'], y=df_daily_agg_aug['NetA_imp'],text=df_daily_agg_aug['NetA_imp'],
            textposition='auto',marker=dict(
        color='rgba(100, 78, 139, 0.6)',
        line=dict(color='rgba(100, 78, 139, 1.0)', width=3))
                            ),
            go.Bar(name="NetB_imp",
            x=df_daily_agg_aug['Date'], y=df_daily_agg_aug['NetB_imp'],text=df_daily_agg_aug['NetB_imp'],
            textposition='auto',marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
                  
                  ),          
            
        ])

fig.update_layout(title="Acquired Impressions on networks For August 2019",
    yaxis_title="Impressions (Sum Total)",xaxis_title='2019',barmode='stack')
fig.show()

### Q1 - Average Network Spend

In [138]:
fig = go.Figure(data=[go.Bar(name="NetA_spd",
            x=df_plot['Weekday'], y=df_plot['NetA_spd'],text=df_plot['NetA_spd'],
            textposition='auto',marker=dict(
        color='rgba(100, 78, 139, 0.6)',
        line=dict(color='rgba(100, 78, 139, 1.0)', width=3))
                            ),
            go.Bar(name="NetB_spd",
            x=df_plot['Weekday'], y=df_plot['NetB_spd'],text=df_plot['NetB_spd'],
            textposition='auto',marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
                  
                  ),          
            
        ])

fig.update_layout(title="Average Network Spend during the week",xaxis_title="Weekdays",
    yaxis_title="Spend",barmode='stack')
fig.show()

### Q1 - Acquired Network Impressions of A and B(Average)

In [139]:
fig = go.Figure(data=[go.Bar(name="NetA_imp",
            x=df_plot['Weekday'], y=df_plot['NetA_imp'],text=df_plot['NetA_imp'],
            textposition='auto',marker=dict(
        color='rgba(10, 78, 139, 0.6)',
        line=dict(color='rgba(10, 78, 139, 1.0)', width=3))
                            ),
            go.Bar(name="NetB_imp",
            x=df_plot['Weekday'], y=df_plot['NetB_imp'],text=df_plot['NetB_imp'],
            textposition='auto',marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
                  
                  ),          
            
        ])

fig.update_layout(title="Average Network Impressions of Net A vs Net B during the week",
    yaxis_title="Impressions",barmode='group')
fig.show()

In [140]:
df_daily_agg.head()

,Date,Page Views,NetA_imp,NetA_spd,NetB_imp,NetB_spd
0,2019-07-07,3363,421,159,381,176
1,2019-07-08,4160,392,1066,316,774
2,2019-07-09,3619,756,877,671,5010
3,2019-07-10,3474,128,130,401,492
4,2019-07-11,3547,154,1994,182,230


### Q2 - Explorary Analysis of Feature relationships

In [141]:
fig = px.scatter_matrix(df_daily_agg, dimensions=['Page Views', 'NetA_imp','NetA_spd'\
                                         , 'NetB_imp', 'NetB_spd'],color='Date')
fig.update_layout(title="Relationships between Page Views, Network Impressions and Spend variables - Aggregated Daily Sum")
fig.show()

In [142]:
fig = px.scatter_matrix(df_plot, dimensions=['Page Views', 'NetA_imp','NetA_spd'\
                                         , 'NetB_imp', 'NetB_spd'],color='Weekday')


fig.update_layout(title="Relationships between Page Views, Network Impressions and Spend variables - Avg by Weekday")

fig.show()

### Q3 - Visualise amount of variation of Page views at different parts of the day

In [143]:
df_hour_agg=df.groupby(['Month','Hour'])[['Page Views', 'NetA_imp','NetA_spd'\
                                         , 'NetB_imp', 'NetB_spd']].sum().reset_index()

df_hour_agg.head()

,Month,Hour,Page Views,NetA_imp,NetA_spd,NetB_imp,NetB_spd
0,Aug,0,1126,1,52,0,0
1,Aug,1,763,0,0,0,0
2,Aug,2,470,0,0,0,0
3,Aug,3,350,0,0,0,0
4,Aug,4,312,0,0,0,0


In [144]:
df_hour_agg=df_hour_agg.merge(df_daypart, left_on='Hour', right_on='Hour', how='inner')

In [145]:
cats = [ 'Late Night', 'Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening', 'Night']

cat_type = CategoricalDtype(categories=cats, ordered=False)
df_hour_agg['Day Part'] = df_hour_agg['Day Part'].astype(cat_type)

In [146]:
df_dp_agg = df_hour_agg.groupby(['Month','Day Part'])[['Page Views', 'NetA_imp','NetA_spd'\
                                         , 'NetB_imp', 'NetB_spd']].sum().reset_index()
df_dp_agg.head()

,Month,Day Part,Page Views,NetA_imp,NetA_spd,NetB_imp,NetB_spd
0,Aug,Late Night,0,0,0,0,0
1,Aug,Early Morning,1610,38,1068,0,0
2,Aug,Morning,0,0,0,0,0
3,Aug,Afternoon,15734,142,2986,260,6228
4,Aug,Late Afternoon,10291,305,4866,184,4798


In [147]:
fig = px.line(df_dp_agg, x='Day Part', y='Page Views', color = "Month", hover_name="Day Part")
fig.update_layout(title='Page views during the day for July and August 2019', xaxis_title='',yaxis_title="Page Views(Total)")
fig.add_trace(go.Scatter(x=df_dp_agg['Day Part'], y=df_dp_agg['Page Views'],
                    mode='markers',name='Marker'))
fig.show()

### Read data files - data Jul 07 - Jul 20 - Part1.xlsx

In [148]:
df_july_p1 = pd.read_excel('/home/shiva/Documents/UTS/02_Courses/DVN/DVN/AT2 Part1/Data/data Jul 07 - Jul 20 - Part1.xlsx')


del df_july_p1['Unnamed: 7']

In [149]:
df_july_p1.head()

,Date,Network,Channel,Contribution,Impression,Site Visits,Spend
0,2019-07-07,NetA,ChannelA1,0.0,0,0.246914,0
1,2019-07-07,NetA,ChannelA1,0.0,0,0.259259,0
2,2019-07-07,NetA,ChannelA1,0.0,0,0.111111,0
3,2019-07-07,NetA,ChannelA1,0.0,0,0.074074,0
4,2019-07-07,NetA,ChannelA1,0.0,0,0.098765,0


In [150]:
### Read data files - data Jul 07 - Jul 20 - Part2.xlsx

In [151]:
df_july_p2 = pd.read_excel('/home/shiva/Documents/UTS/02_Courses/DVN/DVN/AT2 Part1/Data/data Jul 07 - Jul 20 - Part2.xlsx')
df_july_p2


,Hour,Weekday,Month,Market,Creative,Country,Product
0,0,Sun,Jul,ADE,Creative1,AU,Product1
1,1,Sun,Jul,ADE,Creative1,AU,Product1
2,2,Sun,Jul,ADE,Creative1,AU,Product1
3,3,Sun,Jul,ADE,Creative1,AU,Product1
4,4,Sun,Jul,ADE,Creative1,AU,Product1
...,...,...,...,...,...,...,...
136075,19,Sat,Jul,SYD,Creative3,AU,Product5
136076,20,Sat,Jul,SYD,Creative3,AU,Product5
136077,21,Sat,Jul,SYD,Creative3,AU,Product5
136078,22,Sat,Jul,SYD,Creative3,AU,Product5


In [152]:
df_july=pd.merge(df_july_p1, df_july_p2, left_index=True, right_index=True)

In [153]:
df_july

,Date,Network,Channel,Contribution,Impression,Site Visits,Spend,Hour,Weekday,Month,Market,Creative,Country,Product
0,2019-07-07,NetA,ChannelA1,0.0,0,0.246914,0,0,Sun,Jul,ADE,Creative1,AU,Product1
1,2019-07-07,NetA,ChannelA1,0.0,0,0.259259,0,1,Sun,Jul,ADE,Creative1,AU,Product1
2,2019-07-07,NetA,ChannelA1,0.0,0,0.111111,0,2,Sun,Jul,ADE,Creative1,AU,Product1
3,2019-07-07,NetA,ChannelA1,0.0,0,0.074074,0,3,Sun,Jul,ADE,Creative1,AU,Product1
4,2019-07-07,NetA,ChannelA1,0.0,0,0.098765,0,4,Sun,Jul,ADE,Creative1,AU,Product1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136075,2019-07-20,NetB,ChannelB3,0.0,0,2.049383,0,19,Sat,Jul,SYD,Creative3,AU,Product5
136076,2019-07-20,NetB,ChannelB3,0.0,0,2.037037,0,20,Sat,Jul,SYD,Creative3,AU,Product5
136077,2019-07-20,NetB,ChannelB3,0.0,0,1.925926,0,21,Sat,Jul,SYD,Creative3,AU,Product5
136078,2019-07-20,NetB,ChannelB3,0.0,0,1.888889,0,22,Sat,Jul,SYD,Creative3,AU,Product5


### Read data files - data Jul 21 - Aug 03 - Part2.xlsx

In [154]:
df_aug = pd.read_excel('/home/shiva/Documents/UTS/02_Courses/DVN/DVN/AT2 Part1/Data/data Jul 21 - Aug 03.xlsx')
df_aug

,Date,Network,Channel,Contribution,Impression,Site Visits,Spend,Hour,Weekday,Month,Market,Creative,Country,Product
0,2019-07-21,NetA,ChannelA2,0.0,0,0.375000,0,0,Sun,Jul,ADE,Creative2,AU,Product1
1,2019-07-21,NetA,ChannelA2,0.0,0,0.285714,0,1,Sun,Jul,ADE,Creative2,AU,Product1
2,2019-07-21,NetA,ChannelA2,0.0,0,0.321429,0,2,Sun,Jul,ADE,Creative2,AU,Product1
3,2019-07-21,NetA,ChannelA2,0.0,0,0.285714,0,3,Sun,Jul,ADE,Creative2,AU,Product1
4,2019-07-21,NetA,ChannelA2,0.0,0,0.160714,0,4,Sun,Jul,ADE,Creative2,AU,Product1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95755,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,19,Sat,Aug,ADE,Creative2,AU,Product5
95756,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,20,Sat,Aug,ADE,Creative2,AU,Product5
95757,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,21,Sat,Aug,ADE,Creative2,AU,Product5
95758,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,22,Sat,Aug,ADE,Creative2,AU,Product5


In [155]:
df_july_aug = df_july.append(df_aug)

In [156]:
df_july_aug

,Date,Network,Channel,Contribution,Impression,Site Visits,Spend,Hour,Weekday,Month,Market,Creative,Country,Product
0,2019-07-07,NetA,ChannelA1,0.0,0,0.246914,0,0,Sun,Jul,ADE,Creative1,AU,Product1
1,2019-07-07,NetA,ChannelA1,0.0,0,0.259259,0,1,Sun,Jul,ADE,Creative1,AU,Product1
2,2019-07-07,NetA,ChannelA1,0.0,0,0.111111,0,2,Sun,Jul,ADE,Creative1,AU,Product1
3,2019-07-07,NetA,ChannelA1,0.0,0,0.074074,0,3,Sun,Jul,ADE,Creative1,AU,Product1
4,2019-07-07,NetA,ChannelA1,0.0,0,0.098765,0,4,Sun,Jul,ADE,Creative1,AU,Product1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95755,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,19,Sat,Aug,ADE,Creative2,AU,Product5
95756,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,20,Sat,Aug,ADE,Creative2,AU,Product5
95757,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,21,Sat,Aug,ADE,Creative2,AU,Product5
95758,2019-08-03,NetA,ChannelA3,0.0,0,0.000000,0,22,Sat,Aug,ADE,Creative2,AU,Product5


In [157]:
df_july_aug_plot=df_july_aug.groupby(['Network','Channel'])[['Site Visits', 'Impression','Spend']].sum().reset_index()

df_july_aug_plot

,Network,Channel,Site Visits,Impression,Spend
0,NetA,ChannelA1,27106.074074,4090,0
1,NetA,ChannelA2,44395.231481,2870,39200
2,NetA,ChannelA3,32445.583333,655,8630
3,NetA,ChannelA4,44395.231481,8900,163550
4,NetA,ChannelA5,28006.060185,1875,15595
5,NetB,ChannelB1,44395.231481,3885,10645
6,NetB,ChannelB2,31878.356481,7125,11590
7,NetB,ChannelB3,44395.231481,8615,227180


In [158]:
df_july_aug_plot['PCT_Spend']=(df_july_aug_plot['Spend'] / df_july_aug_plot['Spend'].sum()) * 100

In [159]:
df_july_aug_plot['PCT_Impression']=(df_july_aug_plot['Impression'] / df_july_aug_plot['Impression'].sum()) * 100

In [160]:
df_july_aug_plot

,Network,Channel,Site Visits,Impression,Spend,PCT_Spend,PCT_Impression
0,NetA,ChannelA1,27106.074074,4090,0,0.000000,10.758911
1,NetA,ChannelA2,44395.231481,2870,39200,8.228552,7.549651
2,NetA,ChannelA3,32445.583333,655,8630,1.811541,1.723004
3,NetA,ChannelA4,44395.231481,8900,163550,34.331115,23.411811
4,NetA,ChannelA5,28006.060185,1875,15595,3.273578,4.932264
5,NetB,ChannelB1,44395.231481,3885,10645,2.234514,10.219650
6,NetB,ChannelB2,31878.356481,7125,11590,2.432881,18.742602
7,NetB,ChannelB3,44395.231481,8615,227180,47.687819,22.662107


### Q4 - Visualise the percentage variation of spend and impressions across all channels for the period 07/07/2019 to 03/08/2019

In [161]:
fig = go.Figure(data=[go.Bar(name="%Spend",
            x=df_july_aug_plot['Channel'], y=df_july_aug_plot['PCT_Spend'].round(0),text=df_july_aug_plot['PCT_Spend'].round(0),
            textposition='auto',marker=dict(
        color='rgba(10, 78, 139, 0.6)',
        line=dict(color='rgba(10, 78, 139, 1.0)', width=3))
                            ),
            go.Bar(name="%Impression",
            x=df_july_aug_plot['Channel'], y=df_july_aug_plot['PCT_Impression'].round(0),text=df_july_aug_plot['PCT_Impression'].round(0),
            textposition='auto',marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
                  
                  ),          
            
        ])

fig.update_layout(title="% Variation of Spend and Impressions across channels for July and Aug 2019",
    yaxis_title="Percentage(%)",xaxis_title="Channel",barmode='group')
fig.show()